# Unit-test traces

author: laquitainesteeve@gmail.com

Note:

- I can't preprocess the entire traces on the ubuntu machine likely due to lack of RAM.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import spikeinterface as si
from random import choices
from scipy import stats
import spikeinterface.preprocessing as spre

# SET PROJECT PATH

PROJ_PATH = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(PROJ_PATH)

from src.nodes.utils import standardize_layers
from src.nodes.prepro.run import run as prep

RAW_PATH_ns = 'dataset/00_raw/recording_npx_spont'
#PREP_PATH_ns = 'dataset/01_intermediate/preprocessing/recording_npx_spont'
#PREP_PATH_ms = 'dataset/01_intermediate/preprocessing/recording_marques_smith'
#PREP_PATH_e = 'dataset/01_intermediate/preprocessing/recording_npx_evoked'

lyrs = ["L1", "L2/3", "L4", "L5", "L6"]

tmp_path = './' # can be deleted to save space

# parallel processing parameters
N_JOBS = 10                # (default = 20) number of parallel processes for waveform extraction (20/32)
CHUNKS = 10000             # (default = 800000) use smaller chunks to use less RAM

Expected:


Marques-Smith: 362  
Marques-Smith (all sites): 362  
Biophy. spont.: 361  
Biophy. spont. (all sites): 361  
Biophy. evoked: 387  
Biophy. evoked (all sites): 387  

## Custom functions

In [2]:
def report_amplitudes(traces, sites, label):
    """
    Reports the maximum absolute amplitude for the given traces and sites.

    Parameters:
    traces (numpy.ndarray): The traces data.
    sites (numpy.ndarray): The indices of the sites to consider.
    label (str): A label to describe the data being reported.

    Returns:
    None
    """
    max_amplitude_sites = np.max(np.absolute(traces[:, sites]))
    max_amplitude_all = np.max(np.absolute(traces))
    print(f"{label}: {max_amplitude_sites}")
    print(f"{label} (all sites): {max_amplitude_all}")

def prepare_amplitude_data(rec, layers):
    """
    Prepares the data required for amplitude reporting.

    Parameters:
    rec (spikeinterface.RecordingExtractor): The recording extractor object.
    layers (list): List of layer names to filter.

    Returns:
    tuple: A tuple containing traces and site indices.
    """
    traces = rec.get_traces()
    site_layers = rec.get_property("layers")
    site_layers = standardize_layers(site_layers)
    sites = np.where(np.isin(site_layers, layers))[0]
    return traces, sites

In [3]:
%%time 

# load recording
RecNS = si.load_extractor(RAW_PATH_ns)
RecNS = spre.astype(RecNS, "int16")
RecNS = prep(RecNS, freq_min=300, freq_max=None)

2025-07-22 14:37:53,546 - root - run.py - run - INFO - High-pass filtered in 0.01 secs
2025-07-22 14:37:53,546 - root - run.py - run - INFO - Pipeline completed in 0.01 secs
CPU times: user 9.72 ms, sys: 516 µs, total: 10.2 ms
Wall time: 20.8 ms


/home/steeve/steeve/epfl/code/spikebias/envs/spikebias/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(


## Amplitudes

In [ ]:
%%time 

# NS
traces_ns, sites_ns = prepare_amplitude_data(RecNS, lyrs)
report_amplitudes(traces_ns, sites_ns, "Biophy. spont.")

# # Marques-Smith
# RecMS = si.load_extractor(PREP_PATH_ms)
# traces_ms, sites_ms = prepare_amplitude_data(RecMS, lyrs)
# report_amplitudes(traces_ms, sites_ms, "Marques-Smith")

# # E
# RecE = si.load_extractor(PREP_PATH_e)
# traces_e, sites_e = prepare_amplitude_data(RecE, lyrs)
# report_amplitudes(traces_e, sites_ms, "Evoked")